# Nafisur Rahman
nafisur21@gmail.com<br>
https://www.linkedin.com/in/nafisur-rahman

## Finding The most similar sentences to a given sentence on Washington post article.

### 1. Retrieve Text
* Download and parse the text from webpage https://www.washingtonpost.com/news/the-switch/wp/2016/10/18/the-pentagons-massive-new-telescope-is-designed-to-track-space-junk-and-watch-out-for-killer-asteroids.<br>
web scraping using BeatifulSoup

In [1]:
# For a given article url
articleURL = "https://www.washingtonpost.com/news/the-switch/wp/2016/10/18/the-pentagons-massive-new-telescope-is-designed-to-track-space-junk-and-watch-out-for-killer-asteroids/"

In [2]:
import urllib
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from string import punctuation
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict
def getTextfromurl(url):
    page = urllib.request.urlopen(url).read().decode('utf8')
    soup = BeautifulSoup(page,"lxml")
    text = ' '.join(map(lambda p: p.text, soup.find_all('article')))
    return text.encode('ascii', errors='replace').decode('utf-8').replace("?"," ")


In [3]:
text=getTextfromurl(articleURL)
text

"      The Space Surveillance Telescope offers improvements in determining the orbits of newly discovered objects and provides rapid observations of events that may only occur over a relatively short period of time, like a supernova. (DARPAtv)    There are a lot of rocks flying around through space. Lots of debris, too. Old satellites, spent rocket boosters, even for a short while a spatula that got loose during a space shuttle mission in 2006. All of it swirling around in orbit, creating a bit of a traffic jam. For years, the Pentagon has been worried about the collisions that might be caused by an estimated 500,000 pieces of debris, taking out enormously valuable satellites and, in turn, creating even more debris. On Tuesday, the Defense Department took another significant step toward monitoring all of the cosmic junk swirling around in space, by delivering a gigantic new telescope capable of seeing small objects from very far away. Developed by the Defense Advanced Research Project 

In [4]:
from nltk.tokenize import sent_tokenize
sent_list=sent_tokenize(text)
sent_list

['      The Space Surveillance Telescope offers improvements in determining the orbits of newly discovered objects and provides rapid observations of events that may only occur over a relatively short period of time, like a supernova.',
 '(DARPAtv)    There are a lot of rocks flying around through space.',
 'Lots of debris, too.',
 'Old satellites, spent rocket boosters, even for a short while a spatula that got loose during a space shuttle mission in 2006.',
 'All of it swirling around in orbit, creating a bit of a traffic jam.',
 'For years, the Pentagon has been worried about the collisions that might be caused by an estimated 500,000 pieces of debris, taking out enormously valuable satellites and, in turn, creating even more debris.',
 'On Tuesday, the Defense Department took another significant step toward monitoring all of the cosmic junk swirling around in space, by delivering a gigantic new telescope capable of seeing small objects from very far away.',
 'Developed by the Defen

In [5]:
import numpy as np
import pandas as pd
import sklearn
import nltk
import matplotlib.pyplot as plt
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
import re


In [6]:

def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None

In [7]:
def doc_to_synsets(doc):
    text=str(doc)
    text=re.sub('[^a-zA-Z]',' ',text)
    text=word_tokenize(text)
    #text=[w for w in text if w not in Cstopwords]
    #text=[stemmer.stem(w) for w in text if w not in Cstopwords]
    #text=[lemma.lemmatize(w) for w in text if w not in Cstopwords]
    tags=pos_tag(text)
    synset_list=[]
    for tag in tags:
        try:
            ss=wn.synsets(tag[0],pos=convert_tag(tag[1]))[0]
            synset_list.append(ss)
        except:
            None
    return synset_list

In [8]:
def similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2

    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.

    Args:
        s1, s2: list of synsets from doc_to_synsets

    Returns:
        normalized similarity score of s1 onto s2

    Example:
        synsets1 = doc_to_synsets('I like cats')
        synsets2 = doc_to_synsets('I like dogs')
        similarity_score(synsets1, synsets2)
        Out: 0.73333333333333339
    """
    
    list_of_largest_similarities = []
    # Your Code Here
    for syn1 in s1:
        
        largest_sim = 0
        
        for syn2 in s2:
            
            #print(syn1.path_similarity(syn2))
            
            if syn1.path_similarity(syn2):
            
                if syn1.path_similarity(syn2) > largest_sim:
                
                    largest_sim = syn1.path_similarity(syn2)
                
        list_of_largest_similarities.append(largest_sim)
    
    return sum(list_of_largest_similarities) / len(list_of_largest_similarities)


In [9]:
def document_path_similarity(doc1, doc2):
    """Finds the symmetrical similarity between doc1 and doc2"""

    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)

    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2

In [10]:
doc1='On Tuesday, the Defense Department took another significant step toward monitoring all of the cosmic junk swirling around in space, by delivering a gigantic new telescope capable of seeing small objects from very far away.'

In [11]:
for d in sent_list:
    doc2=d
    score=document_path_similarity(doc1, doc2)
    print(score)

0.28291918454753706
0.32909368728334243
0.24100643130815544
0.27338076993249405
0.4099098219787875
0.2583938212170971
1.0
0.35657213555920453
0.35810413245758077
0.2325311988878232
0.11054427578442358
0.19906392458116595
0.27798388752052544
0.37983914890608605
0.3190036187394363
0.2524754519905382
0.26319059945627343
0.3680179634868449
0.23677433677433676
0.390921530576703
0.24138187602627262
0.20792487684729066
0.21076681938750905
0.22573012343127288
0.32045794563035945
0.13226378028102165
0.16900508495336083
0.2946399249354914
0.274365763546798


In [13]:
ranking = defaultdict(int)
for i,sent in enumerate(sent_list):
    score=document_path_similarity(doc1, sent)
    ranking[i] += score
    
ranking

defaultdict(int,
            {0: 0.28291918454753706,
             1: 0.32909368728334243,
             2: 0.24100643130815544,
             3: 0.27338076993249405,
             4: 0.4099098219787875,
             5: 0.2583938212170971,
             6: 1.0,
             7: 0.35657213555920453,
             8: 0.35810413245758077,
             9: 0.2325311988878232,
             10: 0.11054427578442358,
             11: 0.19906392458116595,
             12: 0.27798388752052544,
             13: 0.37983914890608605,
             14: 0.3190036187394363,
             15: 0.2524754519905382,
             16: 0.26319059945627343,
             17: 0.3680179634868449,
             18: 0.23677433677433676,
             19: 0.390921530576703,
             20: 0.24138187602627262,
             21: 0.20792487684729066,
             22: 0.21076681938750905,
             23: 0.22573012343127288,
             24: 0.32045794563035945,
             25: 0.13226378028102165,
             26: 0.1690050849

In [17]:
def similar(question,text, n):
    sents = sent_tokenize(text)
    
    assert n <= len(sents)
    
    
    ranking = defaultdict(int)
    
    for i,sent in enumerate(sent_list):
        score=document_path_similarity(question, sent)
        ranking[i] += score
             
        
    sents_idx = nlargest(n, ranking, key=ranking.get)
    return [sents[j] for j in (sents_idx)]

In [18]:
question='What are the significant steps taken by Defense Department for monitoring cosmic junk?'
text=getTextfromurl(articleURL)
similar(question,text,4)

['On Tuesday, the Defense Department took another significant step toward monitoring all of the cosmic junk swirling around in space, by delivering a gigantic new telescope capable of seeing small objects from very far away.',
 'For years, the Pentagon has been worried about the collisions that might be caused by an estimated 500,000 pieces of debris, taking out enormously valuable satellites and, in turn, creating even more debris.',
 'It is also being used by NASA to monitor asteroids and other near-Earth objects that could collide with the planet, officials said.',
 'The telescope would join another new space debris tracking technology known as the Space Fence, which is now being built by Bethesda-based Lockheed Martin.']